In [2]:
# !pip install pyarango

In [33]:
# future
from __future__ import annotations

# stdlib
from typing import *
from uuid import UUID
from collections import defaultdict

# third party
import pydantic
from pydantic import BaseModel
import pyArango
from pyArango.connection import *

from nacl.signing import SigningKey

# syft absolute
import syft as sy
from syft.core.common import UID
from syft.lib.python import Dict as SyDict

In [10]:
client = Connection(arangoURL='http://127.0.0.1:51930', username="root", password="somepassword")

In [21]:
if client.hasDatabase("app"):
    db = client["app"]
else:
    db = client.createDatabase(name="app")

In [54]:
if not db.hasCollection("users"):
    db.createCollection(name="users")

In [154]:
db.collections

{'_frontend': ArangoDB collection name: _frontend, id: 1541, type: document, status: loaded,
 '_appbundles': ArangoDB collection name: _appbundles, id: 1538, type: document, status: loaded,
 '_apps': ArangoDB collection name: _apps, id: 1535, type: document, status: loaded,
 '_jobs': ArangoDB collection name: _jobs, id: 1532, type: document, status: loaded,
 '_queues': ArangoDB collection name: _queues, id: 1529, type: document, status: loaded,
 '_aqlfunctions': ArangoDB collection name: _aqlfunctions, id: 1526, type: document, status: loaded,
 '_analyzers': ArangoDB collection name: _analyzers, id: 1523, type: document, status: loaded,
 '_graphs': ArangoDB collection name: _graphs, id: 1520, type: document, status: loaded,
 'users': ArangoDB collection name: users, id: 1839, type: document, status: loaded}

In [56]:
class SyftObjectRegistry:
    __object_version_registry__: Dict[str, Dict[int, Type[SyftObject]]] = defaultdict(lambda: {})
    def __init_subclass__(cls, **kwargs: Any) -> None:
        super().__init_subclass__(**kwargs)
        if hasattr(cls, "__canonical_name__"):
            cls.__object_version_registry__[cls.__canonical_name__][int(cls.__version__)] = cls

    @classmethod
    def versioned_class(cls, name: str, version: int) -> Optional[Type[SyftObject]]:
        if name not in cls.__object_version_registry__:
            return None
        classes = cls.__object_version_registry__[name]
        if version not in classes:
            return None
        return classes[version]

In [127]:
class SyftObject(BaseModel, SyftObjectRegistry):    
    class Config:
        arbitrary_types_allowed = True

    # all objects have a UID
    id: UID = None # consistent and persistent uuid across systems
    @pydantic.validator("id", pre=True, always=True)
    def make_id(cls, v):
        return v if isinstance(v, UID) else UID()
    
    __canonical_name__: str # the name which doesn't change even when there are multiple classes
    __version__: int # data is always versioned
    __attr_state__: List[str] # persistent recursive serde keys
    __attr_searchable__: List[str] # keys which can be searched in the ORM
    __attr_unique__: List[str] # the unique keys for the particular Collection the objects will be stored in

    def to_arango(self,doc) -> Dict[str, Any]:
        for k in self.__attr_searchable__:
            doc[k] = getattr(self, k)
        blob = self.to_bytes()
        doc["_id"] = self.id.value
        doc["__canonical_name__"] = self.__canonical_name__
        doc["__version__"] = self.__version__
        doc["__blob__"] = blob
        doc.save()

    @staticmethod
    def from_arango(bson: Any) -> SyftObject:
        constructor = SyftObjectRegistry.versioned_class(
            name=bson["__canonical_name__"], version=bson["__version__"]
        )
        return constructor(**sy.deserialize(bson["__blob__"], from_bytes=True).upcast())

    def to_bytes(self) -> bytes:
        d = SyDict(**self)
        return sy.serialize(d, to_bytes=True)
    
    @staticmethod
    def from_bytes(blob: bytes) -> SyftObject:
        return sy.deserialize(blob, from_bytes=True)

    # allows splatting with **
    def keys(self) -> KeysView[str]:
        return self.__dict__.keys()

    # allows splatting with **
    def __getitem__(self, key: str) -> Any:
        return self.__dict__.__getitem__(key)
    
    def _upgrade_version(self, latest: bool = True) -> SyftObject:
        constructor = SyftObjectRegistry.versioned_class(
            name=self.__canonical_name__, version=self.__version__+1
        )
        if not constructor:
            return self
        else:
            # should we do some kind of recursive upgrades?
            upgraded = constructor._from_previous_version(self)
            if latest:
                upgraded = upgraded._upgrade_version(latest=latest)
            return upgraded

In [128]:
class SyftUser(SyftObject):
    # version
    __canonical_name__ = "SyftUser"
    __version__ = 1

    # fields
    email: str
    name: str
    bad_key: bool = False

    # serde / storage rules
    __attr_state__ = ["email", "name", "bad_key"]
    __attr_searchable__ = ["email", "name", "bad_key"]
    __attr_unique__ = ["email"]

In [129]:
class SyftUserV2(SyftObject):
    # version
    __canonical_name__ = "SyftUser"
    __version__ = 2

    # fields
    email: str
    name: str
    signing_key: bytes

    # serde / storage rules
    __attr_state__ = ["email", "name"]
    __attr_searchable__ = ["email", "name"]
    __attr_unique__ = ["email"]
    
    @classmethod
    def _from_previous_version(cls, userv1: SyftUser) -> SyftUserV2:
        kwargs = dict(**userv1)
        kwargs.update({"signing_key":bytes(SigningKey.generate())})
        return cls(**kwargs) # ignore bad_key

In [130]:
type(client)

pyArango.connection.Connection

In [131]:
# a collection is like a table of documents but with what ever shape you like
class SyftCollection:
    _db: str
    _collection_name: str
    _collection: pyArango.collection.Collection
    _syft_object_type: Dict[int, Type[SyftObject]]
    
    def __init__(self, client: pyArango.connection.Connection) -> None:
        self._db = client[self._db]
        self._collection = self._db[self._collection_name]

    def add(self, obj: SyftObject) -> SyftObject:
        doc = self._collection.createDocument()
        obj.to_arango(doc)

    def drop(self) -> None:
        self._collection.truncate()

    def delete() -> None: pass
    def update() -> None: pass
    def find(self, search_params: Dict[str, Any]) -> List[SyftObject]:
        results = []
        res = self._collection.find(search_params)
        for d in res:
            results.append(SyftObject.from_mongo(d))
        return results
    def find_one(self, search_params: Dict[str, Any]) -> Optional[SyftObject]:
        d = self._collection.find_one(search_params)
        if d is None:
            return d
        return SyftObject.from_mongo(d)

In [132]:
# a collection of SyftUsers
class SyftUserCollection(SyftCollection):
    _db = "app"
    _collection_name = "users"
    __canonical_object_name__ = "SyftUser"

In [133]:
# do some object creation and serde

In [134]:
uid = UUID('3873fc45-f513-48ab-8a47-7306bc7382b0')

In [135]:
madhava = SyftUser(email="madhava@openmined.org", name="Madhava", id=uid)

In [136]:
ser = madhava.to_bytes()

In [137]:
de = SyftUser.from_bytes(ser)

In [138]:
assert madhava == de

In [139]:
key = SigningKey.generate()

In [140]:
madhava_v2 = SyftUserV2(email="madhava@openmined.org", name="Madhava", signing_key=bytes(key))

In [141]:
madhava_v2

SyftUserV2(id=<UID: 1cfacf5b914e49ac9e9a1de539e9ccb7>, email='madhava@openmined.org', name='Madhava', signing_key=b'0uK\x91\xef\tHD\x8e\xc2\x99E\xe1\xf9A#\x0f\x92\xac\xf1\x89=\x02\xf4\xd2\x91\xee\xa7\xce\x11Em')

In [142]:
assert madhava_v2.__canonical_name__ == madhava.__canonical_name__

In [143]:
# do some collection stuff

In [144]:
user_collection = SyftUserCollection(client=client)
user_collection.drop()

In [145]:
user_collection.add(madhava)

UpdateError: invalid document type. Errors: {'code': 400, 'error': True, 'errorMessage': 'invalid document type', 'errorNum': 1227}

In [120]:
user_collection._collection.fetchAll()

In [126]:
user_collection._collection.

ArangoDB collection name: users, id: 1839, type: document, status: loaded

In [112]:
try:
    user_collection.add(madhava)
except pymongo.errors.DuplicateKeyError as e:
    print("Duplicate key")

In [122]:
madhava = user_collection._collection.fetchByExample({"name": "Madhava"},1)

In [123]:
type(madhava)

pyArango.query.SimpleQuery

In [28]:
user_collection.add(madhava_v2)

In [29]:
madhavas = user_collection.find({"name": "Madhava"})

In [30]:
# a collection of different versioned types
upgraded = []
for m in madhavas:
    print(m.__version__, m)
    upgraded.append(m._upgrade_version())
    
for m in upgraded:
    print(m.__version__, m)

1 id=<UID: 9cc591d7c101493c870672f946d14482> email='madhava@openmined.org' name='Madhava' bad_key=False
2 id=<UID: 381f96a1451347d3b49b021ee5e9b770> email='madhava@openmined.org' name='Madhava' signing_key=b'\x96h \xad\xcd\x88\xa9y\x82f\xdc\xfa\xce\xd6\xf5IV.\xfe\x00\xdb%\xff\xd3\xd2_\x19\xbe%e\x17\xb9'
2 id=<UID: 9cc591d7c101493c870672f946d14482> email='madhava@openmined.org' name='Madhava' signing_key=b'=\x86\xcd\xa2Y{\xfc+\xd5Vs:\xb0q\xe0\xb7\x04\xb0\x11H\x8e\x13+\x1b\x87;\x8e\xa1T\xba\xeca'
2 id=<UID: 381f96a1451347d3b49b021ee5e9b770> email='madhava@openmined.org' name='Madhava' signing_key=b'\x96h \xad\xcd\x88\xa9y\x82f\xdc\xfa\xce\xd6\xf5IV.\xfe\x00\xdb%\xff\xd3\xd2_\x19\xbe%e\x17\xb9'
